In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
import math
import itertools
from IPython.display import Image, display
import shutil
%matplotlib inline
import cv2
from scipy import misc
from imageio import imwrite

In [3]:
input = "data4/"

In [4]:
output = "images3/"

In [5]:
MAX_X_ALLOWED = 1920
MAX_Y_ALLOWED = 1080

###  BUTTON
'NoButton' = 1
'Left' = 2
'Scroll' = 3
'Right' = 4
'Middle' = 5

###  STATE
 'Move' = 1
 'Pressed' = 2
 'Drag' = 3
 'Released' = 4
 'Up' = 5
 'Down' = 6

In [6]:
def getPixelFromState(stateVal):
    if stateVal == 2 or stateVal == 6:
        return 100
    elif stateVal == 4 or stateVal == 5:
        return 180
    elif stateVal == 3:
        return 255
    else:
        return 50

In [7]:
def createImage(df, l, t=1):
    img = np.zeros((MAX_X_ALLOWED, MAX_Y_ALLOWED, 3))
    for index, row in df.iterrows():
        if index == 0:
            continue
        lastRow = df.iloc[index-1,:]
        if row["x"] < MAX_X_ALLOWED and row["y"] < MAX_Y_ALLOWED:
            if lastRow["x"] < MAX_X_ALLOWED and lastRow["y"] < MAX_Y_ALLOWED:
                last_x = int(lastRow["x"])
                last_y = int(lastRow["y"])
                x = int(row["x"])
                y = int(row["y"])
                if last_x != x or last_y != y:
                    if row["state"] == 3:
                        t = 2
                    cv2.line(img, (last_y, last_x), (y, x), thickness=t, lineType=l, 
                             color=(getPixelFromState(row["state"]), 
                                    int(255 * (row["speed"])), int(255 * (row["acceleration"]))))
                    if row["button"] == 2:
                        cv2.circle(img,(y,x), 7, (244, 220, 66), -1)
                    elif row["button"] == 3:
                        cv2.circle(img,(y,x), 7, (98, 244, 65), -1)
                    elif row["button"] == 4:
                        cv2.circle(img,(y,x), 7, (241, 65, 244), -1)
    return img.astype(np.uint8)

In [8]:
users = os.listdir(input)
for u in users:
    sessions = os.listdir(input + u)
    if not os.path.exists(output + u):
        os.makedirs(output + u)
    for sess in sessions:
        path = input + u + "/" + sess
        df = pd.read_csv(path)
        img = createImage(df, cv2.LINE_AA, t=1)
        imwrite(output + u + "/" + sess + ".png", img)
